In [1]:
# clone and install the required packages for the Face Recognition project
!git clone https://github.com/sathishkumar67/Face-Recognition-using-Resnet.git
!mv /kaggle/working/Face-Recognition-using-Resnet/* /kaggle/working/
!pip install --upgrade pip
!pip install -r requirements.txt
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

fatal: destination path 'Face-Recognition-using-Resnet' already exists and is not an empty directory.
mv: cannot stat '/kaggle/working/Face-Recognition-using-Resnet/*': No such file or directory
Looking in indexes: https://download.pytorch.org/whl/cu126


In [2]:
# Import necessary libraries
import os
import random
import numpy as np
import torch
from huggingface_hub import hf_hub_download
from siamese_resnet.utils import unzip_file
from siamese_resnet.dataset import TripletDataset, TripletDatasetGenerator, triplet_collate_fn
from siamese_resnet.model import SiameseResNet
from siamese_resnet.loss import TripletLoss
from siamese_resnet.trainer import train_model

# Define constants
DATASET_REPO_ID = "pt-sk/Face_Recognition_Dataset"
DATA_ROOT = "face_recognition_with_cropped_faces_dataset"
DATASET_FILENAME_IN_REPO = f"{DATA_ROOT}.zip"
DATASET_REPO_TYPE = "dataset"
LOCAL_DIR = os.getcwd()
NUM_CORES = os.cpu_count()
SEED = 42
DATALOADER_PIN_MEMORY_DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set the random seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

# Download the dataset from Hugging Face Hub
# hf_hub_download(repo_id=DATASET_REPO_ID, filename=DATASET_FILENAME_IN_REPO, repo_type=DATASET_REPO_TYPE, local_dir=LOCAL_DIR)

# Unzip the dataset
# unzip_file(os.path.join(LOCAL_DIR, DATASET_FILENAME_IN_REPO), LOCAL_DIR)

2025-05-05 13:49:26.524222: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746452966.842356     224 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746452966.923937     224 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746452967.579210     224 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746452967.579251     224 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746452967.579254     224 computation_placer.cc:177] computation placer alr

In [3]:
# Initialize generator
generator = TripletDatasetGenerator(DATA_ROOT)

# Create splits
splits = generator.create_splits()
    
# Generate triplets for each split
train_triplets = generator.generate_triplets(splits['train'])
val_triplets = generator.generate_triplets(splits['val'])
test_triplets = generator.generate_triplets(splits['test'])

# Create datasets
train_dataset = TripletDataset(train_triplets)
val_dataset = TripletDataset(val_triplets)
test_dataset = TripletDataset(test_triplets)

# Create dataloaders (4 workers for optimal IO)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, 
                        num_workers=NUM_CORES, pin_memory=True, persistent_workers=True,
                        collate_fn=triplet_collate_fn, prefetch_factor=2, pin_memory_device=DATALOADER_PIN_MEMORY_DEVICE)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False,
                        num_workers=NUM_CORES, pin_memory=True, persistent_workers=True,
                        collate_fn=triplet_collate_fn, prefetch_factor=2, pin_memory_device=DATALOADER_PIN_MEMORY_DEVICE)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False,
                        num_workers=NUM_CORES, pin_memory=True, persistent_workers=True,
                        collate_fn=triplet_collate_fn, prefetch_factor=2, pin_memory_device=DATALOADER_PIN_MEMORY_DEVICE)

In [4]:
model = SiameseResNet(embedding_dim=256)
criterion = TripletLoss(margin=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=1e-4, betas=(0.9, 0.999))

history, model = train_model(
    model=model.to(DEVICE),
    device=DEVICE,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    epochs=200,
    patience=20,
    checkpoint_path='best_face_model.pth'
)   

Train Epoch 1: 100%|██████████| 169/169 [00:28<00:00,  5.98batch/s, loss=0.35, lr=0.0003] 



Epoch 001 Summary:
Train Loss: 0.4062 | Val Loss: 0.4083
Learning Rate: 3.00e-04 | Time: 30.6s
Best Val Loss: 0.4083 | Patience Left: 20


Train Epoch 2: 100%|██████████| 169/169 [00:25<00:00,  6.60batch/s, loss=0.414, lr=0.0003]



Epoch 002 Summary:
Train Loss: 0.3717 | Val Loss: 0.3850
Learning Rate: 3.00e-04 | Time: 27.3s
Best Val Loss: 0.3850 | Patience Left: 20


Train Epoch 3: 100%|██████████| 169/169 [00:25<00:00,  6.60batch/s, loss=0.482, lr=0.0003]


...


Epoch 037 Summary:
Train Loss: 0.2073 | Val Loss: 0.2647
Learning Rate: 3.00e-09 | Time: 26.9s
Best Val Loss: 0.2579 | Patience Left: 2


Train Epoch 38: 100%|██████████| 169/169 [00:25<00:00,  6.58batch/s, loss=0.257, lr=3e-9]



Epoch 038 Summary:
Train Loss: 0.2045 | Val Loss: 0.2636
Learning Rate: 3.00e-09 | Time: 27.0s
Best Val Loss: 0.2579 | Patience Left: 1


Train Epoch 39: 100%|██████████| 169/169 [00:25<00:00,  6.60batch/s, loss=0.235, lr=3e-9]



Epoch 039 Summary:
Train Loss: 0.2063 | Val Loss: 0.2643
Learning Rate: 3.00e-09 | Time: 26.9s
Best Val Loss: 0.2579 | Patience Left: 0

Early stopping triggered after 39 epochs!
